In [21]:
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools
from copy import deepcopy as dcopy,copy
from scipy.ndimage import measurements
from collections import defaultdict

In [22]:
# Different approach needed for this one as no obvious end point.
# Looks to have worked well. 

url='https://www.janestreet.com/puzzles/square-run/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]

print(" ".join(x[7:35]))

A queen is located at a1 and wishes to travel to h8 via a series of one or more moves. (These must be legal queen’s moves.) After each move, the numbers on each of the squares change. If the move is between two spaces which sum to a perfect square, every number on the board decreases by 1 after the move. Otherwise, each number decreases by 5. (The queen may stop on a square more than once.) What is the largest sum you can obtain from the squares you visit over each move in your journey? Please send us your sum and your list of moves. Example: a5 , a3 , b3 , d1 , a1 , d4 , h8 , a1 , h8 would have a sum of 0 + 26 + 29 + 12 – 7 + 17 + 20 – 10 + 18 = 105.


### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2017/09/20170905_square_run.png" width="350">

In [24]:
# Setup the constraints
fixed = np.rot90([
    [8,5,13,23,29,15,23,30],
    [17,22,30,3,13,25,2,14],
    [10,15,18,28,2,18,27,6],
    [0,31,1,11,22,7,16,20],
    [12,17,24,26,3,24,25,5],
    [27,31,8,11,19,4,12,21],
    [21,20,28,4,9,26,7,14],
    [1,6,9,19,29,10,16,0]
    ],3)

np.max(fixed)

31

In [53]:
class Matrix():
    def __init__(self,fixed):
        self.fixed = fixed
        squares = []
        #for the answer given 0 is a perfect square
        for i in range(0,32):
            squares.append(i**2)
        self.squares = set(squares)
      
    #########################################
    # define possible moves 
    def poss_moves(self,x,y):        
        temp = []
        for i in range(1,8):
            temp.append([x-i,y+i])
            temp.append([x  ,y+i])
            temp.append([x+i,y+i])
            temp.append([x-i,y  ])
            temp.append([x+i,y  ])
            temp.append([x-i,y-i])
            temp.append([x  ,y-i])
            temp.append([x+i,y-i])
        
        ans = []
        for c in temp:
            if np.min(c) < 0 or np.max(c) >7:
                continue
            ans.append(c)
        return ans

   
    ##############################################
    # Turn the grid moves into a nice format
    def print_route(self,route):
        letters =["a","b","c","d","e","f","g","h"]
        out = ""
        for r,c in route:
            out += letters[r]+str(c+1)+","
        return out

                         
    ###############################################
    # Main solver.  
    def solve(self):
        start = time.perf_counter()
        poss_route = [[[[0,0]], 0, 0 ]]  #Route, Score, Points deducted
        max_num = 0
        scores = defaultdict(int)
        while len(poss_route) > 0 :
            route,score,deduct = poss_route.pop()
            g = dcopy(self.fixed) - deduct 
            row,col = route[-1]
        
            if scores[((row,col),deduct)] <= score:
                scores[((row,col),deduct)]  = score 
                start_num = g[row,col]
                end = g[7,7]
                poss_move = self.poss_moves(row,col)
            
               # if you can jump to the end do so
                if score + end> max_num:
                    if [7,7] in poss_move:
                        max_route = route + [[7,7]]
                        max_num = score + end
                        #print("Score = {} Route:{}".format(max_num,self.print_route(max_route)))

                poss_num =[]
            
                for r,c in poss_move:
                    poss_num.append([[r,c],g[r,c]])
            
                poss_num.sort(key=lambda x:x[1],reverse=True)               
        
                for x,new_num in poss_num:
                    r,c = x
                    total = new_num + start_num
                    if total < 0:
                        continue
                    if total in self.squares:
                        new_deduct = deduct + 1
                    else:
                        new_deduct = deduct + 5

                    x = [route + [[r,c]], score+new_num, new_deduct]
                
                    poss_route.insert(0,x)
                    
        print("Solved in {:.4f} seconds".format(time.perf_counter() - start))        
        print("Final Score = {} Route:{}".format(max_num,self.print_route(max_route)))
        

In [54]:
test = Matrix(fixed)
test.solve()

Solved in 46.61739940000189 seconds
Final Score = 305 Route:a1,c3,c7,d8,d1,g4,h3,a3,a4,e8,f7,b3,a3,e7,e5,b5,c4,f4,d4,f2,a2,c2,c7,d8,g8,e8,h8,d4,h8,


In [58]:
url='https://www.janestreet.com/puzzles/solutions/september-2017-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]

print(" ".join(x[7:18]))

The largest sum possible for the queen’s journey is 305 . It turns out that there are 36 paths that achieve such a score, one of which is: c3, c7, d8, d1, g4, h3, a3, a4, e8, f7, b3, a3, e7, e5, b5, c4, f4, d4, f2, c2, c8, e8, b5, c4, c2, b3, f7, e8, h8 , which scores as 8 + 29 + 21 + 16 + 21 + 16 + 21 + 5 + 21 + 16 + 21 + 16 + 1 + 9 + 17 + 9 + 8 + 9 + 8 + 9 – 7 + 8 + 9 + 1 + 4 + 6 – 1 + 2 + 2 = 305 . Maximum scores from:
